In [5]:
!pip install keras-tcn

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from tcn import TCN
import tensorflow as tf
import os

def create_sequences(X_data, y_data, seq_length):
    X, y = [], []
    for i in range(len(X_data) - seq_length):
        X.append(X_data[i:i+seq_length])
        y.append(y_data[i + seq_length])
    return np.array(X), np.array(y)

def compile_model(model):
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

def get_callbacks():
    return [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
    ]

def train_model(model, X_train, y_train, X_val, y_val, name, epochs=100):
    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=epochs, batch_size=32,
                        callbacks=get_callbacks(),
                        verbose=0)
    return model, history

def build_lstm(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    return compile_model(model)

def build_gru(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        GRU(64),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    return compile_model(model)

def build_tcn(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        TCN(nb_filters=64, kernel_size=3, dilations=[1, 2, 4, 8], activation='relu', use_skip_connections=True),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    return compile_model(model)

def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true) + 1e-8))

def wape(y_true, y_pred):
    return 100 * np.sum(np.abs(y_pred - y_true)) / (np.sum(np.abs(y_true)) + 1e-8)

def evaluate_model(model, X_val, y_val, scaler_y):
    y_pred = model.predict(X_val, verbose=0)
    y_pred_inv = scaler_y.inverse_transform(y_pred)
    y_val_inv = scaler_y.inverse_transform(y_val)

    rmse = np.sqrt(mean_squared_error(y_val_inv, y_pred_inv))
    mae = mean_absolute_error(y_val_inv, y_pred_inv)
    mape = mean_absolute_percentage_error(y_val_inv, y_pred_inv)
    smape_score = smape(y_val_inv, y_pred_inv)
    wape_score = wape(y_val_inv, y_pred_inv)
    r2 = r2_score(y_val_inv, y_pred_inv)

    return rmse, mae, mape, smape_score, wape_score, r2, y_val_inv, y_pred_inv

url = 'https://raw.githubusercontent.com/marcelowf/LineNex.Dataset/main/dataset_normalizado.csv'
df = pd.read_csv(url, parse_dates=['Timestamp'])

features = ['Temperature_C', 'Vibration_Hz', 'Power_Consumption_kW', 'Network_Latency_ms',
            'Packet_Loss_%', 'Quality_Control_Defect_Rate_%', 'Predictive_Maintenance_Score',
            'Error_Rate_%', 'Efficiency_Status']
target = 'Production_Speed_units_per_hr'

SEQ_LENGTH = 40
all_results = []

output_dir = 'saved_models'
os.makedirs(output_dir, exist_ok=True)

print("Starting training for machines 1 to 50...")

for machine_id in range(1, 51):
    print(f"\nProcessing Machine_ID: {machine_id}")
    df_machine = df[df['Machine_ID'] == machine_id].copy()

    if df_machine.empty:
        print(f"No data for Machine_ID: {machine_id}. Skipping.")
        continue

    data_to_scale = df_machine[features + [target]].dropna()

    if data_to_scale.empty:
        print(f"No valid data to scale for Machine_ID: {machine_id}. Skipping.")
        continue

    X_raw = data_to_scale[features].values
    y_raw = data_to_scale[[target]].values

    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X_raw)
    y_scaled = scaler_y.fit_transform(y_raw)

    if len(X_scaled) <= SEQ_LENGTH:
        print(f"Not enough data points for sequence creation for Machine_ID: {machine_id}. Skipping.")
        continue

    X_seq, y_seq = create_sequences(X_scaled, y_scaled, SEQ_LENGTH)

    if len(X_seq) < 2:
        print(f"Not enough sequences to split for Machine_ID: {machine_id}. Skipping.")
        continue

    split_idx = int(len(X_seq) * 0.8)
    X_train, X_val = X_seq[:split_idx], X_seq[split_idx:]
    y_train, y_val = y_seq[:split_idx], y_seq[split_idx:]

    if X_train.shape[0] == 0 or X_val.shape[0] == 0:
        print(f"Train or validation set is empty for Machine_ID: {machine_id}. Skipping.")
        continue

    models_to_train = {
        'LSTM': build_lstm,
        'GRU': build_gru,
        'TCN': build_tcn
    }

    for model_name, build_func in models_to_train.items():
        print(f"  Training {model_name} model for Machine_ID {machine_id}...")
        tf.keras.backend.clear_session()

        model = build_func(X_train.shape[1:])
        trained_model, history = train_model(model, X_train, y_train, X_val, y_val, model_name)

        rmse, mae, mape, smape_score, wape_score, r2, _, _ = evaluate_model(trained_model, X_val, y_val, scaler_y)

        final_loss = history.history['loss'][-1]
        final_val_loss = history.history['val_loss'][-1]
        final_mae = history.history['mae'][-1]
        final_val_mae = history.history['val_mae'][-1]
        final_lr = history.history['lr'][-1] if 'lr' in history.history else model.optimizer.learning_rate.numpy()

        model_filename = os.path.join(output_dir, f"model_{model_name}_machine_{machine_id}.h5")
        trained_model.save(model_filename)

        all_results.append({
            'Machine_ID': machine_id,
            'Model': model_name,
            'RMSE': rmse,
            'MAE': mae,
            'MAPE': mape,
            'SMAPE': smape_score,
            'WAPE': wape_score,
            'R2': r2,
            'loss': final_loss,
            'mae': final_mae,
            'val_loss': final_val_loss,
            'val_mae': final_val_mae,
            'learning_rate': final_lr,
            'model_file': model_filename
        })

results_df = pd.DataFrame(all_results)
csv_filename = 'model_training_results.csv'
results_df.to_csv(csv_filename, index=False)

print(f"\nTraining complete. Results saved to '{csv_filename}'")
print("\nSample of the results:")
print(results_df.head())

Starting training for machines 1 to 50...

Processing Machine_ID: 1
  Training LSTM model for Machine_ID 1...


  Training GRU model for Machine_ID 1...


  Training TCN model for Machine_ID 1...



Processing Machine_ID: 2
  Training LSTM model for Machine_ID 2...


  Training GRU model for Machine_ID 2...


...

  Training TCN model for Machine_ID 49...



Processing Machine_ID: 50
  Training LSTM model for Machine_ID 50...


  Training GRU model for Machine_ID 50...


  Training TCN model for Machine_ID 50...



Training complete. Results saved to 'model_training_results.csv'

Sample of the results:
   Machine_ID Model      RMSE       MAE      MAPE      SMAPE       WAPE  \
0           1  LSTM  0.278718  0.239805  2.579628  53.874514  47.105523   
1           1   GRU  0.280325  0.241162  2.591973  54.049491  47.371952   
2           1   TCN  0.332648  0.270704  3.094311  59.361835  53.175039   
3           2  LSTM  0.287232  0.252254  2.234581  58.104345  51.092618   
4           2   GRU  0.287122  0.251402  2.166959  58.120951  50.920113   

         R2      loss       mae  val_loss   val_mae  learning_rate  \
0 -0.010123  0.081891  0.246626  0.077974  0.240319       0.000063   
1 -0.021802  0.080294  0.243273  0.081553  0.244039       0.000250   
2 -0.438838  0.019760  0.113210  0.116957  0.273004       0.000250   
3  0.006495  0.081363  0.246501  0.082653  0.252588       0.000004   
4  0.007255  0.078840  0.241676  0.083231  0.252772       0.000125   

                             model_fil